# labeling:
1. rep_chain is auth_asym_id
2. aas is label_seq_id

## Labeling in ligand_binding_info.csv
Should be the same (and was taken from https://github.com/podleyan/Near-Hit-Scoring)

## Extract NI pockets

In [8]:
with open('/home/vit/Downloads/LIGYSIS_human_chains_per_lig_MAY_2024.pkl', 'rb') as f:
    NI_binding_sites = pickle.load(f)

with open("/home/vit/Downloads/all_ligs_df_2023_07_19.pkl", 'rb') as f:
    lig_data = pickle.load(f)
ION_IDS = sorted(list(set(lig_data[lig_data.name.str.contains(" ION")].id.tolist() + lig_data.query('n_atoms <= 3').id.tolist() + lig_data[lig_data.name.str.contains(" ion")].id.tolist())))

NI_pockets = set()
for key, binding_sites in NI_binding_sites.items():
    ACC, SEG = key.split('_')
    for ID, POIs in binding_sites.items():
        is_NI = False
        for POI in POIs.keys():
            _, ligand, _, _ = POI.split('_')
            if ligand not in ION_IDS:
                is_NI = True
                break
        if is_NI:
            NI_pockets.add(f'{ACC}_{SEG}_{ID}')

## Extract the NI pockets from df

In [10]:
with open('/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    df = pickle.load(file)
df = df[df['SITE_NAME'].isin(NI_pockets)]
assert df['rep_chain'].nunique() == 2275, "Number of unique chains not consistent with LIGYSIS eval study"

# Chain mapping
Map auth chain id to label chain id using the ligysis df.

In [11]:
import pickle

with open('/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    mapping_df = pickle.load(file)

auth_to_label_chain_mapping = {}
for pdb_id, auth_chain_id, label_chain_id in zip(mapping_df['pdb_id'], mapping_df['auth_asym_id'], mapping_df['struct_asym_id']):
    auth_to_label_chain_mapping[(pdb_id, auth_chain_id)] = label_chain_id

# loop over PDB structures
go over PDB and read input indexes.

In [ ]:
import sys
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import cryptoshow_utils

binding_sites = {}
for protein_id, binding_site in zip(df['rep_chain'], df['aas']):
    protein_id = protein_id.replace('_', '')
    if protein_id not in binding_sites:
        binding_sites[protein_id] = []
    binding_sites[protein_id].append(binding_site)

# Translate from author numbering to mmCIF numbering (zero-based)
VERBOSE = True
translated_binding_sites = {}
for protein_id in binding_sites:
    pdb_id, auth_chain_id = protein_id[:4], protein_id[4:]
    label_chain_id = auth_to_label_chain_mapping[(pdb_id, auth_chain_id)]
    translated_binding_sites[protein_id] = []
    for binding_site in binding_sites[protein_id]:
        translated_binding_site = cryptoshow_utils.map_auth_to_mmcif_numbering(pdb_id, label_chain_id, binding_site, auth=False)
        if VERBOSE:
            # ONLY 35 CASES WITH LENGTH MISMATCH! (out of 8244)
            if len(translated_binding_site[0]) != len(binding_site):
                print(f"Warning: Length mismatch for {protein_id}. Original: {len(binding_site)}, Translated: {len(translated_binding_site[0])}")
            else:
                print(f"Successfully translated binding site for {protein_id}. Length: {len(binding_site)}")
        translated_binding_sites[protein_id].append(translated_binding_site)

# save
save for both the merged pockets for residue-level evaluation, and non-merged pockets for pocket-level evaluation. 

In [13]:
with open("/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_NI_for_pocket_level_evaluation.csv", "w") as file:
    for protein_id in translated_binding_sites:
        for binding_site, sequence in translated_binding_sites[protein_id]:
            file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_site))};{sequence}\n')

In [14]:
with open("/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_NI_without_unobserved.csv", "w") as file:
    for protein_id in translated_binding_sites:
        binding_residues = set()
        for binding_site, sequence in translated_binding_sites[protein_id]:
            binding_residues.update(binding_site)
        file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_residues))};{sequence}\n')

In [15]:
df['rep_chain'].nunique() # 2775

2275